Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [1]:
# Download Instacart data (196 MB)
%cd /content/
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

# Unzip to 681.1 MB of data
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

/content
--2020-09-11 00:06:31--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.77.190
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.77.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  57.2MB/s    in 3.4s    

2020-09-11 00:06:35 (57.2 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2

In [13]:
from os import listdir
import pandas as pd

# Go into the new folder. Still works if you're already there
%cd /content/instacart_2017_05_01

# Make a list of dataframe names from present files
df_names = [file[:-4] for file in listdir() if '._' not in file]
print('Dataframe names =',df_names)

# Reads in all dataframes into df
df = { df_name : pd.read_csv(df_name+'.csv') for df_name in df_names}
print('Loaded Dataframes:',list(df.keys()))

/content/instacart_2017_05_01
Dataframe names = ['order_products__prior', 'products', 'orders', 'aisles', 'order_products__train', 'departments']
Loaded Dataframes: ['order_products__prior', 'products', 'orders', 'aisles', 'order_products__train', 'departments']


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [16]:
# A fancy print loop for my dataframe dictionary
def describe_df():
  print('Dataframes in df:')
  mem_usage = 0
  for key in df.keys():
    print(key+':')
    print('columns:',list(df[key].columns))
    print('values:',list(df[key].iloc[0]))
    print('Shape:',df[key].shape)
    print()
    mem_usage += df[key].memory_usage(index=True, deep=True).sum()
  mem_usage = format(mem_usage, ',d')
  print('\nMemory Usage:',mem_usage)
describe_df()

Dataframes in df:
products:
columns: ['product_id', 'product_name', 'aisle_id', 'department_id']
values: [1, 'Chocolate Sandwich Cookies', 61, 19]
Shape: (49688, 4)

orders:
columns: ['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order']
values: [2539329, 1, 'prior', 1, 2, 8, nan]
Shape: (3421083, 7)

aisles:
columns: ['aisle_id', 'aisle']
values: [1, 'prepared soups salads']
Shape: (134, 2)

departments:
columns: ['department_id', 'department']
values: [1, 'frozen']
Shape: (21, 2)

order_products:
columns: ['order_id', 'product_id', 'add_to_cart_order', 'reordered']
values: [1, 49302, 1, 1]
Shape: (33819106, 4)


Memory Usage: 1,734,589,436


In [15]:
# combine both order_product dataframes and pop the originals
df['order_products'] = pd.concat([df.pop('order_products__train'),df.pop('order_products__prior')])
describe_df()

Dataframes in df:
products:
columns: ['product_id', 'product_name', 'aisle_id', 'department_id']
values: [1, 'Chocolate Sandwich Cookies', 61, 19]
Shape: (49688, 4)

orders:
columns: ['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order']
values: [2539329, 1, 'prior', 1, 2, 8, nan]
Shape: (3421083, 7)

aisles:
columns: ['aisle_id', 'aisle']
values: [1, 'prepared soups salads']
Shape: (134, 2)

departments:
columns: ['department_id', 'department']
values: [1, 'frozen']
Shape: (21, 2)

order_products:
columns: ['order_id', 'product_id', 'add_to_cart_order', 'reordered']
values: [1, 49302, 1, 1]
Shape: (33819106, 4)


Memory Usage: 1,734,589,436


In [43]:
# How many times were the top 10 products ordered by product name?
df['popular_products'] = df['order_products']['product_id'].value_counts().sort_values(ascending=False)

df['popular_products'] = df['product_popularity'][df['product_popularity'] > 2900]
df['popular_products'] = df['popular_products'].reset_index()
df['popular_products'].columns = ['product_id', 'n_orders']
# # top_ten = df['products'][df['products']['product_id'].isin(top_ten_ids['product_id'])]
df['popular_products'] = pd.merge(df['popular_products'], df['products'], on='product_id', how='inner') 
# # product_popularity = product_popularity.set_index(pd.Index(range(1,11)))
df['popular_products'].drop(['aisle_id','department_id'], axis=1, inplace=True)
df['popular_products'] = df['popular_products'][['product_id','product_name','n_orders']]
df['popular_products'].head(10)

,product_id,product_name,n_orders
0,24852,Banana,491291
1,13176,Bag of Organic Bananas,394930
2,21137,Organic Strawberries,275577
3,21903,Organic Baby Spinach,251705
4,47209,Organic Hass Avocado,220877
5,47766,Organic Avocado,184224
6,47626,Large Lemon,160792
7,16797,Strawberries,149445
8,26209,Limes,146660
9,27845,Organic Whole Milk,142813


## Reshape Data Section

- Replicate the lesson code. Remember, if you haven't followed along typing out what we did during lecture, do that now to get more repetition with the syntax.
- Take table 2 (the transpose of table 1) and practice changing it into Tidy format and back again. You should not use the transpose operation anywhere in this code:
 - Table 2 --> Tidy
 - Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [19]:
import pandas as pd
import numpy as np

table1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11], 
     [3,      1]],
    index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
    columns=['treatmenta', 'treatmentb'])

table2 = table1.T

table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


In [20]:
# Probably the right method for table2 -> tidy
table2.rename_axis(index='Treatment', inplace=True)
table2.reset_index(inplace=True)
tidy = table2.melt(id_vars='Treatment',value_vars=table2.columns[1:],
                   var_name='Name',value_name='Value')
tidy = tidy[['Name','Treatment','Value']]
tidy

,Name,Treatment,Value
0,John Smith,treatmenta,NaN
1,John Smith,treatmentb,2.0
2,Jane Doe,treatmenta,16.0
3,Jane Doe,treatmentb,11.0
4,Mary Johnson,treatmenta,3.0
5,Mary Johnson,treatmentb,1.0


In [142]:
# Probably the wrong method for table2 -> tidy
# manually transpose a df
def trans(old_table):
  new_table = pd.DataFrame()
  for i, index in enumerate(old_table.index):
    row = old_table.iloc[i]
    new_table[index] = row
  return new_table

tidy = trans(table2)
tidy.rename_axis(index='Name', inplace=True)
tidy.reset_index(inplace=True)
tidy = tidy.melt(id_vars='Name', value_vars=['treatmenta','treatmentb'],
               var_name='Treatment', value_name='Value')
tidy


,Name,Treatment,Value
0,John Smith,treatmenta,NaN
1,Jane Doe,treatmenta,16.0
2,Mary Johnson,treatmenta,3.0
3,John Smith,treatmentb,2.0
4,Jane Doe,treatmentb,11.0
5,Mary Johnson,treatmentb,1.0


In [21]:
# tidy -> table2

table3 = tidy.pivot_table(values=['Value'],
                          index=['Treatment'],
                          columns=['Name'])
table3.columns = table3.columns.get_level_values(1)
table3.index.name = None
table3

Name,Jane Doe,John Smith,Mary Johnson
treatmenta,16.0,NaN,3.0
treatmentb,11.0,2.0,1.0


In [22]:
import seaborn as sns
flights = sns.load_dataset('flights')
print('Shape',flights.shape)
flights.head()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Shape (144, 3)


,year,month,passengers
0,1949,January,112
1,1949,February,118
2,1949,March,132
3,1949,April,129
4,1949,May,121


In [23]:
# Flights Pivot Table
flights_wide = flights.pivot_table(values='passengers', index='year',columns='month')
flights_wide.head()

month,January,February,March,April,May,June,July,August,September,October,November,December
year,,,,,,,,,,,,
1949,112,118,132,129,121,135,148,148,136,119,104,118
1950,115,126,141,135,125,149,170,170,158,133,114,140
1951,145,150,178,163,172,178,199,199,184,162,146,166
1952,171,180,193,181,183,218,230,242,209,191,172,194
1953,196,196,236,235,229,243,264,272,237,211,180,201


## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [51]:
describe_df()

Dataframes in df:
products:
columns: ['product_id', 'product_name', 'aisle_id', 'department_id']
values: [1, 'Chocolate Sandwich Cookies', 61, 19]
Shape: (49688, 4)

orders:
columns: ['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order']
values: [2539329, 1, 'prior', 1, 2, 8, nan]
Shape: (3421083, 7)

aisles:
columns: ['aisle_id', 'aisle']
values: [1, 'prepared soups salads']
Shape: (134, 2)

departments:
columns: ['department_id', 'department']
values: [1, 'frozen']
Shape: (21, 2)

order_products:
columns: ['order_id', 'product_id', 'add_to_cart_order', 'reordered']
values: [1, 49302, 1, 1]
Shape: (33819106, 4)

popular_products:
columns: ['product_id', 'product_name', 'n_orders']
values: [24852, 'Banana', 491291]
Shape: (2051, 3)


Memory Usage: 1,734,891,002


In [72]:
# Which popular items are ordered latest?
# 'popular_products' is from the top ten challenge above
df['popular_products']

def get_order_time(product_id):
  mean = df['popular_products']['mean_hour_of_day']
  if mean is np.nan:
    order_ids = df['order_products']['order_id'][
      df['order_products']['product_id'] == product_id]
    hours_of_day = df['orders']['order_hour_of_day'][
      df['orders']['order_hour_of_day'].isin(order_ids)]
    mean = hours_of_day.mean()
    print(product_id,'is ordered at',mean)
    return mean
  else:
    return mean


df['popular_products']['mean_hour_of_day'] = np.nan
for i, product_id in enumerate(df['popular_products']['mean_hour_of_day']):
  print(i,product_id)
  mean = get_order_time(product_id)
  df['popular_products'].at[i, 'mean_hour_of_day'] = mean

# df['popular_products']['mean_hour_of_day'] = df['popular_products']['product_id'].apply(get_order_time)
# df['popular_products']
# top_ten = df['products'][df['products']['product_id'].isin(top_ten_ids['product_id'])]
# top_ten = pd.merge(top_ten_ids, df['products'], on ='product_id', how ='inner') 
# top_ten = top_ten.set_index(pd.Index(range(1,11)))
# top_ten.drop(['aisle_id','department_id'], axis=1, inplace=True)
# top_ten = top_ten[['product_id','product_name','n_orders']]
# top_ten

0 nan
1 nan
2 nan
3 nan
4 nan
5 nan
6 nan
7 nan
8 nan
9 nan
10 nan
11 nan
12 nan
13 nan
14 nan
15 nan
16 nan
17 nan
18 nan
19 nan
20 nan
21 nan
22 nan
23 nan
24 nan
25 nan
26 nan
27 nan
28 nan
29 nan
30 nan
31 nan
32 nan
33 nan
34 nan
35 nan
36 nan
37 nan
38 nan
39 nan
40 nan
41 nan
42 nan
43 nan
44 nan
45 nan
46 nan
47 nan
48 nan
49 nan
50 nan
51 nan
52 nan
53 nan
54 nan
55 nan
56 nan
57 nan
58 nan
59 nan
60 nan
61 nan
62 nan
63 nan
64 nan
65 nan
66 nan
67 nan
68 nan
69 nan
70 nan
71 nan
72 nan
73 nan
74 nan
75 nan
76 nan
77 nan
78 nan
79 nan
80 nan
81 nan
82 nan
83 nan
84 nan
85 nan
86 nan
87 nan
88 nan
89 nan
90 nan
91 nan
92 nan
93 nan
94 nan
95 nan
96 nan
97 nan
98 nan
99 nan
100 nan
101 nan
102 nan
103 nan
104 nan
105 nan
106 nan
107 nan
108 nan
109 nan
110 nan
111 nan
112 nan
113 nan
114 nan
115 nan
116 nan
117 nan
118 nan
119 nan
120 nan
121 nan
122 nan
123 nan
124 nan
125 nan
126 nan
127 nan
128 nan
129 nan
130 nan
131 nan
132 nan
133 nan
134 nan
135 nan
136 nan
137 nan
138 na

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [ ]:
##### YOUR CODE HERE #####